# Import

In [1]:
# System
import os
import json
import datetime

# Data processing
import numpy as np

# Plot
import matplotlib.pyplot as plt

# Config

In [2]:
config = {}

# Update config
    # Fpath
config.update({"Path_root": "/Volumes/Expansion/User_Backup/b08209033/111-2_DA"})
config.update({"Path_data": os.path.join(config["Path_root"], "data")})
config.update({"Path_src": os.path.join(config["Path_root"], "src")})
config.update({"Path_img": os.path.join(config["Path_root"], "img")})
    # Fname
config.update({"Fname_true_state": "x_t.npz"})
config.update({"Fname_init_state": "x_init.npz"})
config.update({"Fname_ensemble_init_state": "ensemble_x_init.npz"})

# Output config
os.chdir(config["Path_root"])
with open('config.json', 'w') as outfile:
    json.dump(config, outfile, sort_keys=True)

# Helper function

In [3]:
def jacobian(operator, x0, epsilon = 2e-8):
    J = np.zeros((x0.shape[0], operator(x0).shape[0]), dtype = "f8")
    for i in range(J.shape[0]):
        x = np.copy(x0)
        x[i] += epsilon
        f = operator(x) - operator(x0)
        for j in range(J.shape[1]):
            J[i,j] = f[j]/epsilon
    J = J.T
    return J

# Lorenz96 Governing Equation

In [4]:
def diff_eqs(x):
    """
    Lorenz96 model, governing equation
    
    Detect dimension to deal wtih ensemble data
    """
    if (x.ndim == 2):
        dy = (np.roll(x, shift = -1, axis = 0)-np.roll(x, shift = 2, axis = 0))*np.roll(x, shift = 1, axis = 0) -x + 8
    elif (x.ndim == 3):
        dy = (np.roll(x, shift = -1, axis = 1)-np.roll(x, shift = 2, axis = 1))*np.roll(x, shift = 1, axis = 1) -x + 8
    else:
        dy = None
    return dy

# Nature Run

In [5]:
class NatureRun():
    """
    
    Generate(Read) full true state data, and initial (ensemble) state data for 
    forecast-analysis cycle
    Save above results, or detect above result
    """
    def __init__(self, f, N, dT, Tmax, members,
                 true_fname, true_fpath, 
                 init_fname, init_fpath, 
                 ensemble_init_fname, ensemble_init_fpath):
        self.__true_data = None
        self.__initial_data = None
        self.__ensemble_initial_data = None
        self.__f = f
        self.__ndim = N
        self.__dT = dT
        self.__nT = int(Tmax/dT)+1
        self.__Tmax = dT*int(Tmax/dT)
        self.__time = np.linspace(0, dT*int(Tmax/dT), int(Tmax/dT)+1)
        self.__members = members
        self.__true_data_fname = true_fname
        self.__true_data_fpath = true_fpath
        self.__init_data_fname = init_fname
        self.__init_data_fpath = init_fpath
        self.__ensemble_init_data_fname = ensemble_init_fname
        self.__ensemble_init_data_fpath = ensemble_init_fpath
        return None
    def __dynamical_operator(self, x):
        """
        Doing time integration using Runge-Kutta 5 scheme
        """
        k1 = self.__f(x)
        k2 = self.__f(x + k1*self.__dT/2)
        k3 = self.__f(x + (3*k1+k2)*self.__dT/16)
        k4 = self.__f(x + k3*self.__dT/2)
        k5 = self.__f(x + (-3*k2+6*k3+9*k4)*self.__dT/16)
        k6 = self.__f(x + (k1+4*k2+6*k3-12*k4+8*k5)*self.__dT/7)
        new_x = x + (self.__dT/90)*(7*k1 + 32*k3 + 12*k4 + 32*k5 + 7*k6)
        return new_x
    def __generate_initial_data(self, seed = 9527):
        """
        """
        np.random.seed(seed)
        Tspinup = 100
        scaler = 0.1
        IC = (scaler*np.random.randn(self.__ndim)).reshape(-1, 1).astype('f8')
        for i in range(Tspinup):
            IC = self.__dynamical_operator(IC)
        return IC
    def __generate_ensemble_initial_data(self, seed = 9527):
        """
        """
        np.random.seed(seed)
        deterministic_RNG = np.random.randint(int(1e6), size = self.__members)
        ICs = []
        for i in range(self.__members):
            ICs.append(self.__generate_initial_data(deterministic_RNG[i]))
        ICs = np.array(ICs, dtype = "f8")
        return ICs
    def __generate_true_data(self, seed = 9527):
        """
        """
        x_t = [self.__generate_initial_data(seed)]
        for i in range(self.__nT-1):
            x_t.append(self.__dynamical_operator(x_t[i]))
        x_t = np.array(x_t, dtype = "f8")
        return x_t
    def get_True_data(self):
        return self.__true_data
    def get_Initial_data(self):
        return self.__initial_data
    def get_Ensemble_initial_data(self):
        return self.__ensemble_initial_data
    def get_Governing_equation(self):
        return self.__f
    def get_Dimension(self):
        return self.__ndim
    def get_Time(self):
        return self.__time
    def get_unit_timestep(self):
        return self.__dT
    def get_total_timestep(self):
        return self.__nT
    def initialize(self, seed1 = 1234, seed2 = 5678, seed3 = 9012):
        """
        Detect if file existed, and if format shape is qualified, 
        and do following things
        
        Generate data and save 
        Read data
        """
        print("Simluation time:")
        print(self.__time)
        print()
        
        # True state
        print("True state:")
        fpath = os.path.join(self.__true_data_fpath, self.__true_data_fname)
        if not os.path.exists(fpath):
            x_t = self.__generate_true_data(seed1)
            print(f"Generate new true state.")
            self.__true_data = x_t
            os.chdir(self.__true_data_fpath)
            np.savez(self.__true_data_fname, 
                     data = x_t, 
                     time = self.__time, 
                     space = np.array([self.__ndim, 1]))
            print(f"Store new true state.")
        else:
            ticks = os.path.getmtime(fpath)
            realtime = datetime.datetime.fromtimestamp(ticks)
            os.chdir(self.__true_data_fpath)
            infile = np.load(self.__true_data_fname)
            print(f"True state had been stored.\nLast modified: {realtime}")
            if (np.array_equal(infile["time"], self.__time)):
                x_t = infile["data"]
                self.__true_data = x_t
            else:
                print("Mismatch data shape, generate new true state.")
                x_t = self.__generate_true_data(seed1)
                self.__true_data = x_t
                os.chdir(self.__true_data_fpath)
                np.savez(self.__true_data_fname, 
                         data = x_t, 
                         time = self.__time, 
                         space = np.array([self.__ndim, 1]))
                print(f"Store new true state.")
        print("Shape: ",x_t.shape)
        print();print();
        
        # Init state
        print("Initial state:")
        fpath = os.path.join(self.__init_data_fpath, self.__init_data_fname)
        if not os.path.exists(fpath):
            x_init = self.__generate_initial_data(seed2)
            print(f"Generate new initial state.")
            self.__initial_data = x_init
            os.chdir(self.__init_data_fpath)
            np.savez(self.__init_data_fname, 
                     data = x_init, 
                     time = self.__time, 
                     space = np.array([self.__ndim, 1]))
            print(f"Store new initial state.")
        else:
            ticks = os.path.getmtime(fpath)
            realtime = datetime.datetime.fromtimestamp(ticks)
            os.chdir(self.__init_data_fpath)
            infile = np.load(self.__init_data_fname)
            print(f"Initial state had been stored.\nLast modified: {realtime}")
            if (np.array_equal(infile["time"], self.__time)):
                x_init = infile["data"]
                self.__initial_data = x_init
            else:
                print("Mismatch data shape, generate new initial state.")
                x_init = self.__generate_initial_data(seed2)
                self.__initial_data = x_init
                os.chdir(self.__init_data_fpath)
                np.savez(self.__init_data_fname, 
                         data = x_init, 
                         time = self.__time, 
                         space = np.array([self.__ndim, 1]))
                print(f"Store new initial state.")
        print("Shape: ",x_init.shape)
        print();print();
        
        # Ensemble init state
        print("Ensemble initital state:")
        fpath = os.path.join(self.__ensemble_init_data_fpath, self.__ensemble_init_data_fname)
        if not os.path.exists(fpath):
            ensemble_x_init = self.__generate_ensemble_initial_data(seed3)
            print(f"Generate new ensemble initial state.")
            self.__ensemble_initial_data = ensemble_x_init
            os.chdir(self.__ensemble_init_data_fpath)
            np.savez(self.__ensemble_init_data_fname, 
                     data = ensemble_x_init, 
                     time = self.__time, 
                     space = np.array([self.__members, self.__ndim, 1]))
            print(f"Store new ensemble initial state.")
        else:
            ticks = os.path.getmtime(fpath)
            realtime = datetime.datetime.fromtimestamp(ticks)
            os.chdir(self.__ensemble_init_data_fpath)
            infile = np.load(self.__ensemble_init_data_fname)
            print(f"Ensemble initial state had been stored.\nLast modified: {realtime}")
            if (np.array_equal(infile["time"], self.__time)) and (infile["space"][0]==self.__members):
                ensemble_x_init = infile["data"]
                self.__ensemble_initial_data = ensemble_x_init
            else:
                print("Mismatch data shape, generate new ensemble initial state.")
                ensemble_x_init = self.__generate_ensemble_initial_data(seed3)
                self.__ensemble_initial_data = ensemble_x_init
                os.chdir(self.__ensemble_init_data_fpath)
                np.savez(self.__ensemble_init_data_fname, 
                         data = ensemble_x_init, 
                         time = self.__time, 
                         space = np.array([self.__members,self.__ndim, 1]))
                print(f"Store new ensemble initial state.")
        print("Shape: ",ensemble_x_init.shape)
        print();print();
        
        return (x_t, x_init, ensemble_x_init)

# Observation state & operator

In [6]:
class Observation_data():
    """
    Generate observation data according to true state data, which is
    observation = true_state + error
    However, some modifications may be applied
    1. Time scheme, observation time may vary to true state time
    2. Space scheme, observation location may different to true state location
    3. Error scheme, observation error may vary by its generator
    
    Here, I use a tricky method, that I assumed fully-observation and generate
    fully_observation data within class, but the actual get method would be limited by 
    previous scheme
    
    Also, the precision accuracy in python make search algorithm invalid, for 
    the actual time sometimes lost accuracy, a round-off is applied when dealing
    with time data
    """
    def __init__(self, ref_data, ref_time):
        self.__reference_data = ref_data
        self.__reference_time = np.round(ref_time, 5)
        self.__observation_data = None
        self.__observation_error = None
        self.__observation_time = None
        self.__observation_time_filter = None
        self.__observation_space = None
        self.__observation_space_filter = None
        self.__observation_Rstd = None
    def __observation_time_scheme(self, opt = 1):
        """
        Define sub function, employ "time structure" of observation data
        Scheme:
        Full coverage (opt==1): each background correspond to one observation
        Sparse coverage (opt==2): 50% background correspond to one observation, and observation are equally spaced
        Initial coverage (opt==3): Only first 30% background correspond to one observation
        """
        def full_time():
            """
            Time structure of observation data
            All-time observation
            """
            _time = np.round(self.__reference_time, 5)
            _filter = np.full(self.__reference_time.shape, True)
            return _time, _filter
        def sparse_time():
            """
            Time structure of observation data
            Adjacent-spaced observation
            """
            _time = np.round(self.__reference_time, 5)[::2]
            _filter = np.full(self.__reference_time.shape, False)
            _filter[::2] = True
            return _time, _filter
        def initial_time():
            """
            Time structure of observation data
            Only first 30% observation of all simulations
            """
            _time = np.round(self.__reference_time, 5)[:int(len(self.__reference_time)*.3)]
            _filter = np.full(self.__reference_time.shape, False)
            _filter[:int(len(self.__reference_time)*.3)] = True
            return _time, _filter
        
        # Choose scheme
        if (opt == 1):
            _time, _filter = full_time()
        elif (opt == 2):
            _time, _filter = sparse_time()
        elif (opt == 3):
            _time, _filter = initial_time()
        else:
            _time, _filter = None, None
        self.__observation_time = _time
        self.__observation_time_filter = _filter
        return None
    def __observation_space_scheme(self, opt = 1):
        """
        Define sub function, employ "space structure" of observation data
        Scheme:
        Full coverage (opt==1): each location correspond to one observation at valid time
        Sparse coverage (opt==2): 50% location correspond to one observation, and locations are cyclic at valid time
        Initial coverage (opt==3): Only 50% background correspond to one observation, purely random at valid time
        """
        def full_space():
            """
            All locations with observations at valid time
            """
            _space = np.repeat(np.arange(40).reshape(1,-1,1), len(self.__reference_time), axis = 0)
            _filter = np.full(self.__reference_data.shape, True)
            return _space, _filter
        def regular_sparse_space(sparse = 20):
            """
            Half-sparse locations with observations at valid time
            """
            _space = np.repeat(np.arange(40).reshape(1,-1,1), len(self.__reference_time), axis = 0)
            _filter = np.full(self.__reference_data.shape, False)
            ndim = self.__reference_data.shape[1]
            for i in range(self.__reference_data.shape[0]):
                observation_range = ((i*sparse)%ndim ,((i+1)*sparse-1)%ndim+1)
                if (observation_range[0] > observation_range[1]):
                    _space[i,:observation_range[1],:] = -1
                    _space[i,observation_range[0]:,:] = -1
                    _filter[i,:observation_range[1],:] = True
                    _filter[i,observation_range[0]:,:] = True
                else:
                    _space[i,observation_range[0]:observation_range[1],:] = -1
                    _filter[i,observation_range[0]:observation_range[1],:] = True
            return _space, _filter
        def random_sparse_space(percent = 0.5, seed = len(self.__reference_time)):
            """
            Random-sparse locations with observations at valid time
            """
            np.random.seed(seed)
            deterministic_RNG = np.random.randint(int(1e6), size = len(self.__observation_time))
            
            _filter = np.full(self.__reference_data.shape, True)
            _space = np.repeat(np.arange(40).reshape(1,-1,1), len(self.__reference_time), axis = 0)
            for i in range(len(self.__observation_time)):
                np.random.seed(deterministic_RNG[i])
                _filter[i] = np.random.choice(a = [False, True], size = (self.__reference_data.shape[1]), p = [1-percent, percent]).reshape(-1,1)
                _space[np.where(np.logical_not(_filter[i]))[0]] = -1
            return _space, _filter
        # Choose scheme
        if (opt == 1):
            _space, _filter = full_space()
        elif (opt == 2):
            _space, _filter = regular_sparse_space()
        elif (opt == 3):
            _space, _filter = random_sparse_space()
        else:
            _space, _filter = None, None
        self.__observation_space = _space
        self.__observation_space_filter = _filter
        return None
    def __reset_observation(self):
        """
        Reset observation to reference state, which is
        observation = ObsOp(reference) = reference, at valid time
        """
        self.__observation_data = self.__reference_data
        return self.__reference_data
    def __random_error_scheme(self, std, opt = 1, seed = 9527):
        """
        Define sub function employ observation error following specific characteristic
        Scheme
        Pure RNG (opt==1): using scaled white noise
        """
        def PRNG(std, seed = 9527):
            """
            Generate error by white noise
            """
            # Deterministic outer seed
            np.random.seed(seed)
            error = np.random.normal(loc = 0, scale = std, size = self.__observation_data.shape)
            R = np.eye(self.__reference_data.shape[1])*std**2
            return (error, R)
        # Reset std
        self.__observation_Rstd = std
        # Choose scheme
        if (opt == 1):
            error, R = PRNG(std = std, seed = seed)
        else:
            error = None
            R = None
        self.__observation_error = error
        self.__observation_R = R
        return error
    def __fill_observation_error(self, error):
        """
        Adding observation error to observation data
        """
        self.__observation_data = self.__observation_data + error
        return self.__observation_data
    def generate(self, time_scheme = 1, space_scheme = 1, error_scheme = 1, std = 1):
        """
        Find valid time with observation
        Generate observation operator
        Reset observation
        Generate observation error
        Generate observation
        *** Compile sequence matters, dont move it!
        """
        self.__observation_time_scheme(opt = time_scheme)
        self.__observation_space_scheme(opt = space_scheme)
        self.__reset_observation()
        error = self.__random_error_scheme(std, opt = error_scheme)
        self.__fill_observation_error(error)
        return None
    def __check_valid_time(self, time = None):
        """
        Return true if given time correspond to observations, else false
        """
        return np.any(self.__observation_time == time)
    def get_Operator(self, time = None, safe = False):
        """
        Return observation operator at given time if possible
        """
        def abstract_Operator(H, x):
            return H@x
        def instance_Operator(_filter):
            return lambda x: abstract_Operator(np.eye(self.__reference_data.shape[1])[_filter.flatten()], x)
        
        # Single-time
        if (isinstance(time, (int, float))):
            time = round(time, 5)
            if (self.__check_valid_time(time) or safe):
                idx = int(np.argwhere(self.__reference_time == time))
                space_filter = self.__observation_space_filter[idx]
                return instance_Operator(space_filter)
            else:
                return None
        # Several-time
        elif (isinstance(time, tuple)):
            time = (round(time[0], 5), round(time[1], 5))
            # Find observation between
            idx = np.logical_and(time[0] <= self.__observation_time, self.__observation_time <= time[1])
            # Check emptyness
            if (not idx.any()):
                return None
            else:
                idx = self.__observation_time[idx]
                return [self.get_Operator(i) for i in idx]
        else:
            return None
    def get_Data(self, time = None):
        """
        Return observation data at given time if possible
        """
        # Single-time
        if (isinstance(time, (int, float))):
            time = round(time, 5)
            if self.__check_valid_time(time):
                # Find closest observation
                idx = int(np.argwhere(self.__reference_time == time))
                return self.get_Operator(time, safe = True)(self.__observation_data[idx])
            else:
                return None
        # Several-time
        elif (isinstance(time, tuple)):
            time = (round(time[0], 5), round(time[1], 5))
            # Find observation between
            idx = np.logical_and(time[0] <= self.__observation_time, self.__observation_time <= time[1])
            # Check emptyness
            if (not idx.any()):
                return None
            else:
                idx = self.__observation_time[idx]
                
                return [self.get_Data(i) for i in idx]
        else:
            return None
    def get_Covariance(self, time):
        """
        Return observation covariance at given time if possible
        """
        # Single-time
        if (isinstance(time, (int, float))):
            time = round(time, 5)
            if self.__check_valid_time(time):
                idx = int(np.argwhere(self.__reference_time == time))
                return np.eye(np.sum(self.__observation_space_filter[idx]))*self.__observation_Rstd**2
            else:
                return None
        # Several-time
        elif (isinstance(time, tuple)):
            time = (round(time[0], 5), round(time[1], 5))
            # Find observation between
            idx = np.logical_and(time[0] <= self.__observation_time, self.__observation_time <= time[1])
            # Check emptyness
            if (not idx.any()):
                return None
            else:
                idx = self.__observation_time[idx]
                
                return [self.get_Covariance(i) for i in idx]
        else:
            return None
    def get_Trajectory(self, loc):
        """
        Non-filtered observation trajectory
        """
        return (self.__observation_data[:,loc,:], self.__observation_space_filter[:,loc,:])
    def get_Error(self):
        return self.__observation_error
    def get_Time(self):
        return self.__observation_time
    def get_Rstd(self):
        return self.__observation_Rstd

# Data Assimilation

## No DA

In [7]:
class Simulator_NoDA():
    def __init__(self,
                 nature_run,
                 observation):
        self.__observation = observation
        self.__initial_x_a = nature_run.get_Initial_data()
        self.__initial_x_b = np.full(self.__initial_x_a.shape, np.nan, dtype = "f8")
        self.__B = np.eye(nature_run.get_Dimension())*observation.get_Rstd()**2
        self.__func = nature_run.get_Governing_equation()
        self.__dT = nature_run.get_unit_timestep()
        self.__nT = nature_run.get_total_timestep()
        return None
    def __model(self, x):
        """
        Doing time integration using Runge-Kutta 4 scheme
        """
        k1 = self.__func(x)
        k2 = self.__func(x + k1*self.__dT/2)
        k3 = self.__func(x + (3*k1+k2)*self.__dT/16)
        k4 = self.__func(x + k3*self.__dT/2)
        k5 = self.__func(x + (-3*k2+6*k3+9*k4)*self.__dT/16)
        k6 = self.__func(x + (k1+4*k2+6*k3-12*k4+8*k5)*self.__dT/7)
        new_x = x + (self.__dT/90)*(7*k1 + 32*k3 + 12*k4 + 32*k5 + 7*k6)
        return new_x
    def Cycle(self):
        observation = self.__observation
        x_b_all = self.__initial_x_b[np.newaxis]
        x_a_all = self.__initial_x_a[np.newaxis]
        ct = 0
        while ct < (self.__nT-1):
            # Forecast
            x_b = self.__model(x_a_all[ct])[np.newaxis]
            x_b_all = np.vstack([x_b_all, x_b])
            # Analysis
            x_b = x_b_all[ct+1]
            y_o = observation.get_Data((ct+1)*self.__dT)
            R = observation.get_Covariance((ct+1)*self.__dT)
            H = observation.get_Operator((ct+1)*self.__dT)
            x_a = x_b[np.newaxis]
            x_a_all = np.vstack([x_a_all, x_a])
            # next
            ct += 1
        return x_a_all

## OI

In [8]:
class Simulator_OI():
    """
    Implement forecast-analysis cycle using OI scheme
    """
    def __init__(self,
                 nature_run,
                 observation):
        self.__observation = observation
        self.__initial_x_a = nature_run.get_Initial_data()
        self.__initial_x_b = np.full(self.__initial_x_a.shape, np.nan, dtype = "f8")
        self.__B = np.eye(nature_run.get_Dimension())*observation.get_Rstd()**2
        self.__Bi = np.linalg.inv(self.__B)
        self.__func = nature_run.get_Governing_equation()
        self.__dT = nature_run.get_unit_timestep()
        self.__nT = nature_run.get_total_timestep()
        return None
    def __model(self, x):
        """
        Doing time integration using Runge-Kutta 5 scheme
        """
        k1 = self.__func(x)
        k2 = self.__func(x + k1*self.__dT/2)
        k3 = self.__func(x + (3*k1+k2)*self.__dT/16)
        k4 = self.__func(x + k3*self.__dT/2)
        k5 = self.__func(x + (-3*k2+6*k3+9*k4)*self.__dT/16)
        k6 = self.__func(x + (k1+4*k2+6*k3-12*k4+8*k5)*self.__dT/7)
        new_x = x + (self.__dT/90)*(7*k1 + 32*k3 + 12*k4 + 32*k5 + 7*k6)
        return new_x
    def __OI(self, x_b, y_o, H, R):
        """
        Data assimilation using optimal interpolation
        
        x_a = x_b + K@[y_o - H(x_b)]
        K = A@H.T@R-1
        A-1 = B-1 + H.T@R-1@H
        """
        d = y_o - H(x_b)
        Hl = jacobian(H, x_b)
        Ri = np.linalg.inv(R)
        A = np.linalg.inv(self.__Bi + Hl.T@Ri@Hl)
        K = A@Hl.T@Ri
        x_a = x_b + K@d
        return x_a
    def Cycle(self):
        """
        Forecast-Analysis Cycle
        """
        observation = self.__observation
        x_b_all = self.__initial_x_b[np.newaxis]
        x_a_all = self.__initial_x_a[np.newaxis]
        ct = 0
        while ct < (self.__nT-1):
            # Forecast
            x_b = self.__model(x_a_all[ct])[np.newaxis]     
            x_b_all = np.vstack([x_b_all, x_b])
            # Analysis
            x_b = x_b_all[ct+1]
            y_o = observation.get_Data((ct+1)*self.__dT)
            H = observation.get_Operator((ct+1)*self.__dT)
            R = observation.get_Covariance((ct+1)*self.__dT)
            if y_o is not None:
                x_a = self.__OI(x_b, y_o, H, R)[np.newaxis]
            else:
                x_a = x_b[np.newaxis]
            x_a_all = np.vstack([x_a_all, x_a])
            # next
            ct += 1
        return x_a_all
    def NMC(self, x_a_all, inflation = 0.25):
        """
        Derive statistical BEC using NMC
        In this case, following Prof's advice, using
        Long forecast = 8 timestep
        Short forecast = 4 timestep
        By deduct different forecast at same valid time, expect the BEC structure
        can be revealed.
        """
        # Algorithm parameter
        long_forecast_length = 8
        short_forecast_length = 4
        forecast_length = long_forecast_length + 1
        N_samples = np.shape(x_a_all)[0]
        # Init
        samples = np.zeros((N_samples, forecast_length, np.shape(x_a_all)[-2], 1))
        
        # Forecast in each sampled analysis time
        for idx_samples in range(N_samples):
            current_x_b = x_a_all[idx_samples]
            samples[idx_samples,0] = current_x_b
            for idx_forecast in range(forecast_length-1):
                # Forecast
                new_x_b = self.__model(current_x_b)
                current_x_b = new_x_b
                
                samples[idx_samples,idx_forecast+1] = current_x_b
                
        # Deduct different forecast length at same valid time
        samples_diff = samples[:-(long_forecast_length-short_forecast_length),long_forecast_length] - samples[(long_forecast_length-short_forecast_length):,short_forecast_length]
        # Calculate statistical BEC
        NMC_B = np.zeros((np.shape(x_a_all)[-2], np.shape(x_a_all)[-2]))
        for i in range(len(samples_diff)):
            d = samples_diff[i]
            NMC_B += (d@d.T/(np.shape(samples_diff)[0]-1))
        # Inflate
        NMC_B *= inflation
        return NMC_B
    def set_B(self, B):
        self.__B = B
        self.__Bi = np.linalg.inv(self.__B)
        return None
    def get_B(self):
        return self.__B

## 3DVar

In [9]:
class Simulator_3DVar():
    """
    Implement forecast-analysis cycle using 3DVar scheme
    """
    def __init__(self,
                 nature_run,
                 observation):
        self.__observation = observation
        self.__initial_x_a = nature_run.get_Initial_data()
        self.__initial_x_b = np.full(self.__initial_x_a.shape, np.nan, dtype = "f8")
        self.__B = np.eye(nature_run.get_Dimension())*observation.get_Rstd()**2
        self.__Bi = np.linalg.inv(self.__B)
        self.__func = nature_run.get_Governing_equation()
        self.__dT = nature_run.get_unit_timestep()
        self.__nT = nature_run.get_total_timestep()
        return None
    def __model(self, x):
        """
        Doing time integration using Runge-Kutta 5 scheme
        """
        k1 = self.__func(x)
        k2 = self.__func(x + k1*self.__dT/2)
        k3 = self.__func(x + (3*k1+k2)*self.__dT/16)
        k4 = self.__func(x + k3*self.__dT/2)
        k5 = self.__func(x + (-3*k2+6*k3+9*k4)*self.__dT/16)
        k6 = self.__func(x + (k1+4*k2+6*k3-12*k4+8*k5)*self.__dT/7)
        new_x = x + (self.__dT/90)*(7*k1 + 32*k3 + 12*k4 + 32*k5 + 7*k6)
        return new_x
    def __increment_3DVar(self, x_b, y_o, H, Ri):
        """
        Data assimilation using 3DVar, in this case, using incremental form
        """
        def f(dx, x_a, x_b, d, Hl, Ri):
            """
            Cost function of 3DVar in incremental form
            J = 1/2*(dx.T@B-1@dx + (H@dx - d)@R-1@(H@dx - d))
            """
            # Rename
            increment = dx
            past_state = x_a
            current_state = past_state + increment
            background_state = x_b
            innovation = d
            LinObsOp = Hl
            
            J_b = (1/2)*(current_state - background_state).T@self.__Bi@(current_state - background_state)
            J_o = (1/2)*(LinObsOp@increment - innovation).T@Ri@(LinObsOp@increment - innovation)
            J = J_b + J_o
            return J
        def fprime(dx, x_a, x_b, d, Hl, Ri):
            """
            Jacobian of cost function of 3DVar in incremental form
            dJ = B-1@dx + H.T@R-1@(H@dx - d)
            """
            # Rename
            increment = dx
            past_state = x_a
            current_state = past_state + increment
            background_state = x_b
            innovation = d
            LinObsOp = Hl
            
            J_b = self.__Bi@(current_state - background_state)
            J_o = (LinObsOp.T)@Ri@(LinObsOp@increment - innovation)
            J = J_b + J_o
            return J
        def line_search(f, grad, dx, args = ()):
            """
            Modified from Golden Section Line Search
            Given object function, its gradient and reference state
            Find step (p_k) that minimize f, which is
            f(dx + p_k * grad) < f(dx)
            ***May unstable due to CFL condition
            """
            # Algorithm parameter
            upper_bound = 1
            lower_bound = -1
            alpha = (3 - np.sqrt(5)) / 2
            # Initialize
            a0 = lower_bound
            b0 = upper_bound
            a1 = a0 + alpha*(b0-a0)
            b1 = b0 - alpha*(b0-a0)
            reference = f(dx, *args)
            
            # Find best step
            while True:
                result_a1 = f(dx+a1*grad, *args)
                result_b1 = f(dx+b1*grad, *args)
                if (result_a1 < reference):
                    best_p = a1
                    break
                if (result_b1 < reference):
                    best_p = b1
                    break
                if (result_a1 < result_b1):
                    b0 = b1
                    b1 = a1
                    a1 = a0 + alpha*(b0-a0)
                else:
                    a0 = a1
                    a1 = b1
                    b1 = b0 - alpha*(b0-a0)
            return best_p
        def PR_beta(g0, g1):
            """
            Polak-Ribiere method for conjugate gradient method 
            """
            return (g1.T@(g1-g0))/(g0.T@g0)
        def minimize(x_b, y_o, H, Ri):
            """
            Minimize cost funciton using CG method
            """
            # Algorithm parameter
            maxiter_inner = int(1e6)
            maxiter_outer = int(1e6)
            # First guess, x = x_b
            x_a = np.copy(x_b)
            
            for out_loop in range(maxiter_outer):
                # Initial minimization parameter
                jac_old = np.full(np.shape(x_b), 0, dtype = "f8")
                jac_new = np.full(np.shape(x_b), 0, dtype = "f8")
                beta = None
                grad = 0
                # Initial increment
                dx = np.full(np.shape(x_b), 0, dtype = "f8")
                # Find current variable
                d = y_o - H(x_a)
                Hl = jacobian(H, x_a)
                
                # Control parameter
                args = (x_a, x_b, d, Hl, Ri)
                for in_loop in range(maxiter_inner):
                    # Find gradient
                    jac = fprime(dx, *args)
                    jac_new = jac
                    beta = PR_beta(jac_old, jac_new) if in_loop != 0 else 1
                    grad = -(jac_new) + beta*grad
                    jac_old = jac_new
                    # Find directions and step
                    p_k = line_search(f, grad, dx, args = args)
                    delta = p_k*grad
                    # Update dx
                    dx += delta
                    if (np.linalg.norm(delta) < 1e-4):
                        break
                # Update x
                x_a += dx
                if (np.linalg.norm(dx) < 1e-3):
                    break
            return x_a
        return minimize(x_b, y_o, H, Ri)  
        
    def Cycle(self):
        """
        Forecast-Analysis Cycle
        """
        observation = self.__observation
        x_b_all = self.__initial_x_b[np.newaxis]
        x_a_all = self.__initial_x_a[np.newaxis]
        ct = 0
        while ct < (self.__nT-1):
            # Forecast
            x_b = self.__model(x_a_all[ct])[np.newaxis]
            x_b_all = np.vstack([x_b_all, x_b])
            # Analysis
            x_b = x_b_all[ct+1]
            y_o = observation.get_Data((ct+1)*self.__dT)
            H = observation.get_Operator((ct+1)*self.__dT)
            R = observation.get_Covariance((ct+1)*self.__dT)
            if y_o is not None:
                Ri = np.linalg.inv(R)
                x_a = self.__increment_3DVar(x_b, y_o, H, Ri)[np.newaxis]
            else:
                x_a = x_b[np.newaxis]
            
            x_a_all = np.vstack([x_a_all, x_a])
            # next
            ct += 1
        return x_a_all
    def NMC(self, x_a_all, inflation = 0.25):
        """
        Derive statistical BEC using NMC
        In this case, following Prof's advice, using
        Long forecast = 8 timestep
        Short forecast = 4 timestep
        By deduct different forecast at same valid time, expect the BEC structure
        can be revealed.
        """
        # Algorithm parameter
        long_forecast_length = 8
        short_forecast_length = 4
        forecast_length = long_forecast_length + 1
        N_samples = np.shape(x_a_all)[0]
        # Init
        samples = np.zeros((N_samples, forecast_length, np.shape(x_a_all)[-2], 1))
        # Forecast in each sampled analysis time
        for idx_samples in range(N_samples):
            current_x_b = x_a_all[idx_samples]
            samples[idx_samples,0] = current_x_b
            for idx_forecast in range(forecast_length-1):
                # Forecast
                new_x_b = self.__model(current_x_b)
                current_x_b = new_x_b
                samples[idx_samples,idx_forecast+1] = current_x_b
        # Deduct different forecast length at same valid time 
        samples_diff = samples[:-(long_forecast_length-short_forecast_length),long_forecast_length] - samples[(long_forecast_length-short_forecast_length):,short_forecast_length]
        # Calculate statistical BEC
        NMC_B = np.zeros((np.shape(x_a_all)[-2], np.shape(x_a_all)[-2]))
        for i in range(len(samples_diff)):
            d = samples_diff[i]
            NMC_B += (d@d.T/(np.shape(samples_diff)[0]-1))
        # Inflate
        NMC_B *= inflation
        return NMC_B
    def set_B(self, B):
        self.__B = B
        self.__Bi = np.linalg.inv(self.__B)
        return None
    def get_B(self):
        return self.__B

## 4DVar

In [10]:
class Simulator_4DVar():
    """
    Implement forecast-analysis cycle using 4DVar scheme
    """
    def __init__(self,
                 nature_run,
                 observation,
                 assimilation_window):
        self.__observation = observation
        self.__initial_x_a = nature_run.get_Initial_data()
        self.__initial_x_b = np.full(self.__initial_x_a.shape, np.nan, dtype = "f8")
        self.__B = np.eye(nature_run.get_Dimension())*observation.get_Rstd()**2
        self.__Bi = np.linalg.inv(self.__B)
        self.__func = nature_run.get_Governing_equation()
        self.__assimilation_window = assimilation_window
        self.__dT = nature_run.get_unit_timestep()
        self.__nT = nature_run.get_total_timestep()
        return None
    def __model(self, x):
        """
        Doing time integration using Runge-Kutta 5 scheme
        """
        k1 = self.__func(x)
        k2 = self.__func(x + k1*self.__dT/2)
        k3 = self.__func(x + (3*k1+k2)*self.__dT/16)
        k4 = self.__func(x + k3*self.__dT/2)
        k5 = self.__func(x + (-3*k2+6*k3+9*k4)*self.__dT/16)
        k6 = self.__func(x + (k1+4*k2+6*k3-12*k4+8*k5)*self.__dT/7)
        new_x = x + (self.__dT/90)*(7*k1 + 32*k3 + 12*k4 + 32*k5 + 7*k6)
        return new_x
    def __increment_4DVar(self, x_b, y_o, H, R):
        """
        Data assimilation using 4DVar, in this case, using incremental form
        """
        def f(dx, x_a, x_b, d, Hl, M, Ri):
            """
            Cost function of 4DVar in incremental form
            J = 1/2*(dx.T@B-1@dx + np.sum((H@M@dx - d)@R-1(H@M@dx - d)))
            """
            # Rename
            increment = dx
            past_state = x_a
            current_state = past_state + increment
            background_state = x_b
            innovation = d
            LinObsOp = Hl
            LinModelOp = M
            
            J_b = (1/2)*(current_state - background_state).T@self.__Bi@(current_state - background_state)
            J_o = 0
            for i in range(len(d)):
                J_o += (1/2)*(LinObsOp[i]@LinModelOp[i]@increment-innovation[i]).T@Ri[i]@(LinObsOp[i]@LinModelOp[i]@increment-innovation[i])
            J = J_b + J_o
            return J
        def fprime(dx, x_a, x_b, d, Hl, M, Ri):
            """
            Jacobian of cost function of 4DVar in incremental form
            dJ = B-1@dx + np.sum(M@H.T@R-1@(H@M@dx - d))
            """
            # Rename
            increment = dx
            past_state = x_a
            current_state = past_state + increment
            background_state = x_b
            innovation = d
            LinObsOp = Hl
            LinModelOp = M
            
            J_b = self.__Bi@(current_state - background_state)
            J_o = 0
            for i in range(len(y_o)):
                J_o += LinModelOp[i].T@LinObsOp[i].T@Ri[i]@(LinObsOp[i]@LinModelOp[i]@increment-innovation[i])
            
            J = J_b + J_o
            return J
        def line_search(f, grad, dx, args = ()):
            """
            Modified from Golden Section Line Search
            Given object function, its gradient and reference state
            Find step (p_k) that minimize f, which is
            f(dx + p_k * grad) < f(dx)
            ***Still unstable due to CFL condition
            """
            # Algorithm parameter
            upper_bound = 1
            lower_bound = -1
            alpha = (3 - np.sqrt(5)) / 2
            # Initialize
            a0 = lower_bound
            b0 = upper_bound
            a1 = a0 + alpha*(b0-a0)
            b1 = b0 - alpha*(b0-a0)
            reference = f(dx, *args)
            
            # Find best step
            while True:
                result_a1 = f(dx+a1*grad, *args)
                result_b1 = f(dx+b1*grad, *args)
                if (result_a1 < reference):
                    best_p = a1
                    break
                if (result_b1 < reference):
                    best_p = b1
                    break
                if (result_a1 < result_b1):
                    b0 = b1
                    b1 = a1
                    a1 = a0 + alpha*(b0-a0)
                else:
                    a0 = a1
                    a1 = b1
                    b1 = b0 - alpha*(b0-a0)
            return best_p
        def PR_beta(g0, g1):
            """
            Polak-Ribiere method for conjugate gradient method 
            """
            return (g1.T@(g1-g0))/(g0.T@g0)
        def minimize(x_b, y_o, H, R):
            """
            Minimize cost function using CG method
            """
            # Algorithm parameter
            maxiter_inner = int(1e6)
            maxiter_outer = int(1e6)
            # First guess, x = x_b
            x_a = np.copy(x_b)
            for out_loop in range(maxiter_outer):
                # Initial minimization parameter
                jac_old = np.full(np.shape(x_b), 0, dtype = "f8")
                jac_new = np.full(np.shape(x_b), 0, dtype = "f8")
                beta = None
                grad = 0
                
                # Initial increment
                dx = np.full(np.shape(x_b), 0, dtype = "f8")
                # Initial model jacobian
                M = []
                M.append(np.eye(x_b.shape[0])) # The initial element is actually not jacobian
                # Initial model trajectory
                base_trajectory = []
                base_trajectory.append(x_a)
                # Initial observation operator jacobian
                Hl = []
                Hl.append(jacobian(H[0], base_trajectory[0]))
                # Initial innovation
                d = []
                d.append(y_o[0]-H[0](base_trajectory[0]))
                
                # Complete TLM (Tangent Linear Model)
                for i in range(len(y_o)-1):
                    M.append(jacobian(self.__model, base_trajectory[i]))
                    base_trajectory.append(self.__model(base_trajectory[i]))
                    d.append(y_o[i+1]-H[i+1](base_trajectory[i+1]))
                    Hl.append(jacobian(H[i+1], base_trajectory[i+1]))
                M = np.array(M, dtype = "f8")
                base_trajectory = np.array(base_trajectory, dtype = "f8")
                
                # Combine model operator
                for i in range(len(M)-1):
                    M[i+1] = M[i+1]@M[i]
                # 
                Ri = []
                for i in range(len(R)):
                    Ri.append(np.linalg.inv(R[i]))
                
                # Control parameter
                args = (x_a, x_b, d, Hl, M, Ri)
                for in_loop in range(maxiter_inner):
                    # Find gradient
                    jac = fprime(dx, *args)
                    jac_new = jac
                    beta = PR_beta(jac_old, jac_new) if in_loop != 0 else 1
                    grad = -(jac_new) + beta*grad     
                    jac_old = jac_new
                    p_k = line_search(f, grad, dx, args = args)
                    delta = p_k*grad
                    # Update dx
                    dx += delta
                    if (np.linalg.norm(delta) < 1e-4):
                        break
                # Update x
                x_a += dx
                if (np.linalg.norm(dx) < 1e-3):
                    break
            return x_a
        return minimize(x_b, y_o, H, R)  
        
    def Cycle(self):
        """
        Forecast-Analysis Cycle
        """
        observation = self.__observation
        x_b_all = self.__initial_x_b[np.newaxis]
        x_a_all = self.__initial_x_a[np.newaxis]
        ct = 0
        used_observation_time = 0
        while ct < (self.__nT-1):
            
            # print(ct*self.__dT)
            # Forecast
            x_b = self.__model(x_a_all[ct])[np.newaxis]
            x_b_all = np.vstack([x_b_all, x_b])
            # Analysis
            x_b = x_b_all[ct+1]
            if (used_observation_time>=(ct+1)*self.__dT):
                x_a = x_b[np.newaxis]
            else:
                args = ((ct+1)*self.__dT, (ct+self.__assimilation_window)*self.__dT)
                y_o = observation.get_Data(args)
                H = observation.get_Operator(args)
                R = observation.get_Covariance(args)
                if y_o is not None:
                    x_a = self.__increment_4DVar(x_b, y_o, H, R)[np.newaxis]
                    used_observation_time = (ct+self.__assimilation_window)*self.__dT
                else:
                    x_a = x_b[np.newaxis]
            x_a_all = np.vstack([x_a_all, x_a])
            # next
            ct += 1
        return x_a_all

## EKF

In [11]:
class Simulator_EKF():
    """
    Implement forecast-analysis cycle using EKF scheme
    """
    def __init__(self,
                 nature_run,
                 observation):
        self.__observation = observation
        self.__initial_x_a = nature_run.get_Initial_data()
        self.__initial_x_b = np.full(self.__initial_x_a.shape, np.nan, dtype = "f8")
        self.__B = np.eye(nature_run.get_Dimension())*observation.get_Rstd()**2
        self.__func = nature_run.get_Governing_equation()
        self.__dT = nature_run.get_unit_timestep()
        self.__nT = nature_run.get_total_timestep()
        return None
    def __model(self, x):
        """
        Doing time integration using Runge-Kutta 5 scheme
        """
        k1 = self.__func(x)
        k2 = self.__func(x + k1*self.__dT/2)
        k3 = self.__func(x + (3*k1+k2)*self.__dT/16)
        k4 = self.__func(x + k3*self.__dT/2)
        k5 = self.__func(x + (-3*k2+6*k3+9*k4)*self.__dT/16)
        k6 = self.__func(x + (k1+4*k2+6*k3-12*k4+8*k5)*self.__dT/7)
        new_x = x + (self.__dT/90)*(7*k1 + 32*k3 + 12*k4 + 32*k5 + 7*k6)
        return new_x
    def __EKF_forecast(self, x_a, Ba):
        """
        Data assimilation using Extended Kalman Filter
        Forecast step
        """
        Q = 0
        M = jacobian(self.__model, x_a)
        Bf = M@Ba@M.T + Q
        return Bf
    def __EKF_analysis(self, x_b, y_o, H, Bf, R):
        """
        Data assimilation using Extended Kalman Filter
        Analysis step
        """
        d = y_o - H(x_b)
        Hl = jacobian(H, x_b)
        K = Bf@Hl.T@np.linalg.inv(Hl@Bf@Hl.T + R)
        x_a = x_b + K@d
        I = np.eye(len(x_b))
        Ba = (I - K@Hl)@Bf
        
        return x_a, Ba
    def __EKF_MUL_inflation(self, B, alpha = 1):
        return alpha*B
    def __EKF_ADD_inflation(self, Ba, alpha = 0):
        return Ba + np.eye(len(Ba))*alpha
    def __EKF_RTP_inflation(self, Ba, Bf, alpha = 0):
        return (1-alpha)*Ba + alpha*Bf
    def Cycle(self, MUL_param = 1, ADD_param = 0, RTP_param = 0):
        """
        Forecast-Analysis Cycle
        """
        observation = self.__observation
        x_b_all = self.__initial_x_b[np.newaxis]
        x_a_all = self.__initial_x_a[np.newaxis]
        Bf = self.__B
        Ba = self.__B

        ct = 0
        while ct < (self.__nT-1):
            
            # Forecast
            x_b = self.__model(x_a_all[ct])[np.newaxis]
            Bf = self.__EKF_forecast(x_a_all[ct], Ba)
            Bf = self.__EKF_MUL_inflation(Bf, alpha = MUL_param)
            
            x_b_all = np.vstack([x_b_all, x_b]) 
            # Analysis
            x_b = x_b_all[ct+1]
            y_o = observation.get_Data((ct+1)*self.__dT)
            H = observation.get_Operator((ct+1)*self.__dT)
            R = observation.get_Covariance((ct+1)*self.__dT)
            if y_o is not None:
                x_a, Ba = self.__EKF_analysis(x_b, y_o, H, Bf, R)
                x_a = x_a[np.newaxis]
                Ba = self.__EKF_RTP_inflation(Ba, Bf, alpha = RTP_param)
                Ba = self.__EKF_ADD_inflation(Ba, alpha = ADD_param)
            else:
                x_a = x_b[np.newaxis]
                Ba = Bf
            
            x_a_all = np.vstack([x_a_all, x_a])
            #
            ct += 1
        return x_a_all

## PO-EnKF

In [57]:
class Simulator_EnKF():
    """
    Implement forecast-analysis cycle using EnKF scheme
    """
    def __init__(self,
                 nature_run,
                 observation):
        self.__observation = observation
        self.__initial_x_a = nature_run.get_Ensemble_initial_data()
        self.__initial_x_b = np.full(self.__initial_x_a.shape, np.nan, dtype = "f8")
        self.__B = np.eye(nature_run.get_Dimension())*observation.get_Rstd()**2
        self.__Bi = np.linalg.inv(self.__B)
        self.__func = nature_run.get_Governing_equation()
        self.__dT = nature_run.get_unit_timestep()
        self.__nT = nature_run.get_total_timestep()
        return None

    def __model(self, x):
        """
        Doing time integration using Runge-Kutta 5 scheme
        """
        k1 = self.__func(x)
        k2 = self.__func(x + k1*self.__dT/2)
        k3 = self.__func(x + (3*k1+k2)*self.__dT/16)
        k4 = self.__func(x + k3*self.__dT/2)
        k5 = self.__func(x + (-3*k2+6*k3+9*k4)*self.__dT/16)
        k6 = self.__func(x + (k1+4*k2+6*k3-12*k4+8*k5)*self.__dT/7)
        new_x = x + (self.__dT/90)*(7*k1 + 32*k3 + 12*k4 + 32*k5 + 7*k6)
        return new_x
    def __ensemble_mean(self, x):
        """
        Derive ensemble mean
        """
        return np.mean(x, axis = 0)
    def __ensemble_BEC(self, x, x_mean):
        """
        Derive ensemble forecast
        """
        B = np.zeros((x_mean.shape[0], x_mean.shape[0]), dtype = "f8")
        for i in range(len(x)):
            
            B += (x[i]-x_mean)@(x[i]-x_mean).T
        B /= (len(x)-1)
        return B
    def __EnKF_forecast(self, x_a, Ba):
        """
        Data assimilation using Extended Kalman Filter
        Forecast step
        """
        Q = 0
        M = jacobian(self.__model, x_a)
        Bf = M@Ba@M.T + Q
        return Bf
    def __EnKF_analysis(self, x_b, x_b_mean, y_o, H, R, Bf):
        """
        Derive ensemble Kalman gain, and update ensemble state
        PH.T = (x_b - H(x_b_mean))@(x_b - H(x_b_mean)).T
        HPH.T = (H(x_b) - H(x_b_mean))@(H(x_b) - H(x_b_mean)).T
        K = (PH.T)/(HPH.T + R)
        """
        Hl = jacobian(H, x_b_mean)
        K = Bf@Hl.T@np.linalg.inv(Hl@Bf@Hl.T+R)
#         Hx = []
#         for i in range(len(x_b)):
#             Hx.append(H(x_b[i]))
#         Hx = np.array(Hx, dtype = "f8")
#         Hx_mean = self.__ensemble_mean(Hx)
        
#         P_HlT = np.zeros((Bf.shape[0], R.shape[0]), dtype = "f8")
#         for i in range(len(x_b)):
#             P_HlT += (x_b[i]-x_b_mean)@(Hx[i]-Hx_mean).T
#         P_HlT /= (len(x_b)-1)
#         Hl_P_HlT = np.zeros((R.shape[0], R.shape[0]), dtype = "f8")
#         for i in range(len(x_b)):
#             Hl_P_HlT += (Hx[i]-Hx_mean)@(Hx[i]-Hx_mean).T
#         Hl_P_HlT /= (len(x_b)-1)
#         K = P_HlT@np.linalg.inv(Hl_P_HlT+R)
        
        x_a = []
        for i in range(len(x_b)):
            x_a.append(x_b[i]+K@(y_o[i]-H(x_b[i])))
        x_a = np.array(x_a, dtype = "f8")
        
        I = np.eye(len(x_b_mean))
        Ba = (I - K@Hl)@Bf
        
        return x_a, Ba
    def __EnKF_B_localization(self, Bf, L = 5):
        for i in range(Bf.shape[0]):
            for j in range(Bf.shape[1]):
                distance = np.abs(i-j)%20
                Bf[i,j] = Bf[i,j]*np.exp(-(distance**2)/(2*L**2))
        return Bf
    def __EnKF_MUL_inflation(self, B, alpha = 1):
        return alpha*B
    def __EnKF_ADD_inflation(self, Bf, alpha = 0):
        return Bf + np.eye(len(Bf))*alpha
    def __EnKF_RTP_inflation(self, Ba, Bf, alpha = 0):
        return (1-alpha)*Ba + alpha*Bf
    def Cycle(self, Radius = 40, MUL_param = 1, ADD_param = 0, RTP_param = 0):
        """
        Forecast-Analysis Cycle
        """
        observation = self.__observation
        x_b_all = self.__initial_x_b[np.newaxis]
        x_b_mean_all = self.__ensemble_mean(self.__initial_x_b)[np.newaxis]
        x_a_all = self.__initial_x_a[np.newaxis]
        x_a_mean_all = self.__ensemble_mean(self.__initial_x_a)[np.newaxis]
        Bf = self.__ensemble_BEC(x_b_all[-1], x_b_mean_all[-1])
        Ba = self.__ensemble_BEC(x_a_all[-1], x_a_mean_all[-1])
        
        ct = 0
        while ct < (self.__nT-1):
            # print(ct*self.__dT)
            # Forecast
            x_b = self.__model(x_a_all[ct])[np.newaxis]
            x_b_mean = self.__ensemble_mean(x_b[0])[np.newaxis]
            Bf = self.__ensemble_BEC(x_b[0], x_b_mean[0])
            Bf = self.__EnKF_ADD_inflation(Bf, alpha = ADD_param)
            Bf = self.__EnKF_B_localization(Bf, Radius)
            Bf = self.__EnKF_MUL_inflation(Bf, alpha = MUL_param)
            
            x_b_all = np.vstack([x_b_all, x_b])
            x_b_mean_all = np.vstack([x_b_mean_all, x_b_mean])
            
            # Analysis
            x_b = x_b_all[ct+1]
            x_b_mean = x_b_mean_all[ct+1]
            y_o = observation.get_Data((ct+1)*self.__dT)
            H = observation.get_Operator((ct+1)*self.__dT)
            R = observation.get_Covariance((ct+1)*self.__dT)
            if y_o is not None:
                np.random.seed(int(ct*self.__nT*9527))
                virtual_y_o = np.random.multivariate_normal(y_o.flatten(), R*(x_b_all.shape[1]-1)/x_b_all.shape[1], size = x_b_all.shape[1])[:,:,np.newaxis]
                x_a, Ba = self.__EnKF_analysis(x_b, x_b_mean, virtual_y_o, H, R, Bf)
                x_a = x_a[np.newaxis]
                x_a_mean = self.__ensemble_mean(x_a[0])[np.newaxis]
            else:
                x_a = x_b[np.newaxis]
                x_a_mean = x_b_mean[np.newaxis]
                Ba = Bf
            x_a_all = np.vstack([x_a_all, x_a])
            x_a_mean_all = np.vstack([x_a_mean_all, x_a_mean])
            #
            ct += 1
        return (x_b_all, x_b_mean_all), (x_a_all, x_a_mean_all)